In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

STARTING_CASH = 100000
ALLOWED_DRIFT = 0.05
relative_path = 'C:/Users/Tiger/PycharmProjects/BettermentCaseStudy'

returns_df = pd.read_csv(relative_path + '/portfolio_returns.csv', index_col=0, parse_dates=True)
tickers = returns_df.columns
dates = returns_df.index
initial_weights = pd.DataFrame(dict(zip(tickers, [0.25, 0.25, 0.125, 0.125, 0.04, 0.035, 0.125, 0.05])), index=[dates[0]])
initial_weights

,eem,efa,ive,iwn,iws,shy,tip,vti
2004-01-02,0.05,0.125,0.125,0.04,0.035,0.25,0.25,0.125


In [32]:
# NON REBALANCED PORTFOLIO
cum_returns_df = (returns_df+1).cumprod()
non_rebalance_df = (cum_returns_df * STARTING_CASH) * initial_weights.values[0]
non_rebalance_df_alloc = (non_rebalance_df.div(non_rebalance_df.sum(axis=1), axis = 0))
cum_returns_df.head(5)

,shy,tip,vti,ive,iwn,iws,efa,eem
2004-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2004-01-05,1.000148,0.998616,1.011336,1.012441,1.009501,1.009284,1.019006,1.034969
2004-01-06,1.001335,1.004983,1.014670,1.013728,1.006262,1.011605,1.020635,1.023313
2004-01-07,1.002077,1.005813,1.017559,1.014586,1.014468,1.010445,1.010589,1.025767
2004-01-08,1.002226,1.001938,1.021338,1.019734,1.021162,1.013153,1.024980,1.029448


In [21]:
non_rebalance_df_alloc.plot(figsize=(16, 8))
plt.show()
#non_rebalance_df.sum(axis=1).plot(figsize=(16,8))
pass